# More about Null Hypothesis Significance Testing (NHST)

In [1]:
import pandas as pd
import itertools as it
from scipy.stats import binom_test
from numpy.random import RandomState

## How to test whether a coin is fair

![title](part_of_scientific_method.png)

- **Hypothesize:** We have a coin, and we hypothesize that it is biased - the probability of getting heads is not equal to $0.5$.
- **Experiment:** We decide to toss the coin 8 times and record the results. This produces heads $7$ times and tails $1$ time.
- **Test hypothesis:** Our _null hypothesis_ (that we would like to reject) is that the coin is fair, with probability of heads equal to $0.5$. What do we do now?

![title](sig_testing_slide.png)

So let's set $\alpha = 0.05$ (more on this later). **Now we need to compute a $p$-value.**

**Question:** From 8 tosses, our experiment produced 7 heads. Which outcomes, from tossing a coin 8 times, are "your effect (or larger)"?

**Answer:** All the outcomes with $0$ heads, $1$ head, $7$ heads or $8$ heads.

We are doing a _two-tailed_ test because a biased coin is biased whether it over-produces heads or over-produces tails.

**So let's calculate the $p$-value:** the probability of getting 0, 1, 7 or 8 heads from 8 tosses of a _fair_ coin.

In [2]:
num_tosses = 8
tosses_df = pd.DataFrame(list(it.product('HT', repeat=num_tosses)), columns=[f'Toss {i+1}' for i in range(num_tosses)])
tosses_df

,Toss 1,Toss 2,Toss 3,Toss 4,Toss 5,Toss 6,Toss 7,Toss 8
0,H,H,H,H,H,H,H,H
1,H,H,H,H,H,H,H,T
2,H,H,H,H,H,H,T,H
3,H,H,H,H,H,H,T,T
4,H,H,H,H,H,T,H,H
...,...,...,...,...,...,...,...,...
251,T,T,T,T,T,H,T,T
252,T,T,T,T,T,T,H,H
253,T,T,T,T,T,T,H,T
254,T,T,T,T,T,T,T,H


In [3]:
tosses_df.sample(5)

,Toss 1,Toss 2,Toss 3,Toss 4,Toss 5,Toss 6,Toss 7,Toss 8
183,T,H,T,T,H,T,T,T
2,H,H,H,H,H,H,T,H
243,T,T,T,T,H,H,T,T
233,T,T,T,H,T,H,H,T
126,H,T,T,T,T,T,T,H


In [4]:
tosses_df['num_heads'] = tosses_df.apply(lambda row: len([x for x in row if x == 'H']), axis='columns')

In [5]:
tosses_df.sample(5)

,Toss 1,Toss 2,Toss 3,Toss 4,Toss 5,Toss 6,Toss 7,Toss 8,num_heads
98,H,T,T,H,H,H,T,H,5
162,T,H,T,H,H,H,T,H,5
246,T,T,T,T,H,T,T,H,2
104,H,T,T,H,T,H,H,H,5
242,T,T,T,T,H,H,T,H,3


**Question:** How do we get the $p$-value now?

In [6]:
tosses_df['num_heads'].isin([0, 1, 7, 8]).sum() / tosses_df.shape[0]

0.0703125

**So we cannot reject the null hypothesis.** So we cannot conclue that the coin is biased.

## Using `binom_test` from `scipy.stats`

Above we did the process "on foot" for illustration, but fortunately in practise we don't need to; the [Binomial Test](https://en.wikipedia.org/wiki/Binomial_test) is built into `scipy`.

In [7]:
tosses_df['num_heads'].isin([0, 1, 7, 8]).sum() / tosses_df.shape[0]

0.0703125

In [8]:
num_heads = 7
num_tails = 1
binom_test(x = [num_heads, num_tails])

0.0703125

## Gratuitous quokka break:

![title](quokka_break_3.jpg)

## We can use hypothesis tests like classifiers

Suppose we have a big bag of coins, say $10000$, some of them fair and some of them biased.

If our hypothesis test works the way we want, we should be able to use it to **separate the biased coins from the fair coins**.

In [9]:
bag_of_coins = (  # Represent each coin by the probability that it produces heads
    
      [0.5] * 5000  # 5000 fair coins
    + [0.6] * 2000  # 2000 coins biased towards heads
    + [0.4] * 3000  # 3000 coins biased towards tails
)

In [10]:
RandomState(11).choice(bag_of_coins, size=10, replace=False)

array([0.5, 0.6, 0.4, 0.6, 0.6, 0.4, 0.6, 0.5, 0.5, 0.5])

Write a function to test a given coin by experiment:

In [11]:
def test_coin(coin_head_prob, random_state=RandomState(), num_tosses=30, alpha=0.05):
    
    # Toss the coin as many times as required; count how many heads:
    num_heads = (random_state.random_sample(num_tosses) < coin_head_prob).sum()

    num_tails = num_tosses - num_heads    

    p_value = binom_test(x = [num_heads, num_tails])

    if p_value <= alpha:
        return "Biased"
    else:
        return "Fair"

In [12]:
test_coin(0.5, num_tosses=100)

'Fair'

In [13]:
test_coin(0.2, num_tosses=100)

'Biased'

In [14]:
test_coin(0.4, num_tosses=100)

'Fair'

**Let's test the whole bag of coins, and see how well we do separating the biased coins from the fair coins, _in aggregate_:**

In [15]:
def test_bag_of_coins(bag_of_coins, random_state=RandomState(), num_tosses=150, alpha=0.05):

    return pd.DataFrame([
        {
            'actual': "Fair" if coin_head_prob == 0.5 else "Biased",
            'predicted': test_coin(coin_head_prob, random_state=random_state, num_tosses=num_tosses, alpha=alpha)
        }
        for coin_head_prob in bag_of_coins
    ])

In [16]:
test_results_df = test_bag_of_coins(bag_of_coins, random_state=RandomState(17))
test_results_df

,actual,predicted
0,Fair,Fair
1,Fair,Fair
2,Fair,Fair
3,Fair,Fair
4,Fair,Fair
...,...,...
9995,Biased,Fair
9996,Biased,Biased
9997,Biased,Biased
9998,Biased,Biased


In [17]:
def summarise_results(results_df):

    CT = pd.crosstab(results_df['predicted'], results_df['actual'])
    display(CT)

    true_fair_predictions    = CT['Fair'  ]['Fair'  ]
    false_fair_predictions   = CT['Biased']['Fair'  ]
    true_biased_predictions  = CT['Biased']['Biased']
    false_biased_predictions = CT['Fair'  ]['Biased']
    
    fair_predictions   = true_fair_predictions   + false_fair_predictions
    biased_predictions = true_biased_predictions + false_biased_predictions
    
    actual_fair   = true_fair_predictions   + false_biased_predictions
    actual_biased = true_biased_predictions + false_fair_predictions

    print("When we predict 'Fair', we are correct %.1f%% of the time and incorrect %.1f%% of the time" % (
        100 * true_fair_predictions / fair_predictions,
        100 * (1 - true_fair_predictions / fair_predictions)
    ))

    print("When we predict 'Biased', we are correct %.1f%% of the time and incorrect %.1f%% of the time" % (
        100 * true_biased_predictions / biased_predictions,
        100 * (1 - true_biased_predictions / biased_predictions)
    ))

    print("When the coin is 'Fair', we are correct %.1f%% of the time and incorrect %.1f%% of the time" % (
        100 * true_fair_predictions / actual_fair,
        100 * (1 - true_fair_predictions / actual_fair)
    ))

    print("When the coin is 'Biased', we are correct %.1f%% of the time and incorrect %.1f%% of the time" % (
        100 * true_biased_predictions / actual_biased,
        100 * (1 - true_biased_predictions / actual_biased)
    ))
    
    print("We predicted correctly for %.1f%% of coins in the bag" % 
         (100 * (true_fair_predictions + true_biased_predictions) / results_df.shape[0]))
        

In [18]:
summarise_results(test_results_df)

actual,Biased,Fair
predicted,,
Biased,3317,213
Fair,1683,4787


When we predict 'Fair', we are correct 74.0% of the time and incorrect 26.0% of the time
When we predict 'Biased', we are correct 94.0% of the time and incorrect 6.0% of the time
When the coin is 'Fair', we are correct 95.7% of the time and incorrect 4.3% of the time
When the coin is 'Biased', we are correct 66.3% of the time and incorrect 33.7% of the time
We predicted correctly for 81.0% of coins in the bag


**That worked pretty well huh?** The diagonal cells of our confusion matrix are bigger than the off-diagonal ones, and we have 81% accuracy...

## Your turn!

Experiment by changing one or more of the following parameters:

- the threshold $\alpha$
- the number of tosses performed in each experiment
- the balance of biased vs fair coins in the bag
- the level of bias in the biased coins

Please play fair and don't scroll down the notebook yet...

In [19]:
your_bag_of_coins = (
      [0.5] * 5000  # 5000 fair coins
    + [0.6] * 2000  # 2000 coins biased towards heads
    + [0.4] * 3000  # 3000 coins biased towards tails
)

In [20]:
summarise_results(test_bag_of_coins(
    your_bag_of_coins,
    random_state=RandomState(17),
    num_tosses=150,
    alpha=0.05
))

actual,Biased,Fair
predicted,,
Biased,3317,213
Fair,1683,4787


When we predict 'Fair', we are correct 74.0% of the time and incorrect 26.0% of the time
When we predict 'Biased', we are correct 94.0% of the time and incorrect 6.0% of the time
When the coin is 'Fair', we are correct 95.7% of the time and incorrect 4.3% of the time
When the coin is 'Biased', we are correct 66.3% of the time and incorrect 33.7% of the time
We predicted correctly for 81.0% of coins in the bag


## Some more examples

**Making the number of coin tosses in the experiments small diminishes our ability to classify the coins:**

In [21]:
summarise_results(test_bag_of_coins(
    bag_of_coins,
    random_state=RandomState(17),
    num_tosses=30
))

actual,Biased,Fair
predicted,,
Biased,854,222
Fair,4146,4778


When we predict 'Fair', we are correct 53.5% of the time and incorrect 46.5% of the time
When we predict 'Biased', we are correct 79.4% of the time and incorrect 20.6% of the time
When the coin is 'Fair', we are correct 95.6% of the time and incorrect 4.4% of the time
When the coin is 'Biased', we are correct 17.1% of the time and incorrect 82.9% of the time
We predicted correctly for 56.3% of coins in the bag


**Conversely making the number of coin tosses very large improves our ability to separate the biased coins from the fair coins:**

In [22]:
summarise_results(test_bag_of_coins(
    bag_of_coins,
    random_state=RandomState(17),
    num_tosses=500
))

actual,Biased,Fair
predicted,,
Biased,4970,224
Fair,30,4776


When we predict 'Fair', we are correct 99.4% of the time and incorrect 0.6% of the time
When we predict 'Biased', we are correct 95.7% of the time and incorrect 4.3% of the time
When the coin is 'Fair', we are correct 95.5% of the time and incorrect 4.5% of the time
When the coin is 'Biased', we are correct 99.4% of the time and incorrect 0.6% of the time
We predicted correctly for 97.5% of coins in the bag


**Making the biased coins more extremely biased makes the task easier:**

In [23]:
extreme_bias_bag_of_coins = (
      [0.5] * 5000
    + [0.8] * 2000  # This probability was 0.6 originally
    + [0.2] * 3000  # This probability was 0.4 originally
)

In [24]:
summarise_results(test_bag_of_coins(
    extreme_bias_bag_of_coins,
    random_state=RandomState(17)
))

actual,Biased,Fair
predicted,,
Biased,5000,213
Fair,0,4787


When we predict 'Fair', we are correct 100.0% of the time and incorrect 0.0% of the time
When we predict 'Biased', we are correct 95.9% of the time and incorrect 4.1% of the time
When the coin is 'Fair', we are correct 95.7% of the time and incorrect 4.3% of the time
When the coin is 'Biased', we are correct 100.0% of the time and incorrect 0.0% of the time
We predicted correctly for 97.9% of coins in the bag


**Making the biased coins barely biased makes the task hard:**

In [25]:
slight_bias_bag_of_coins = (
      [0.5] * 5000
    + [0.55] * 2000  # This probability was 0.6 originally
    + [0.48] * 3000  # This probability was 0.2 originally
)

In [26]:
summarise_results(test_bag_of_coins(
    slight_bias_bag_of_coins,
    random_state=RandomState(17)
))

actual,Biased,Fair
predicted,,
Biased,613,213
Fair,4387,4787


When we predict 'Fair', we are correct 52.2% of the time and incorrect 47.8% of the time
When we predict 'Biased', we are correct 74.2% of the time and incorrect 25.8% of the time
When the coin is 'Fair', we are correct 95.7% of the time and incorrect 4.3% of the time
When the coin is 'Biased', we are correct 12.3% of the time and incorrect 87.7% of the time
We predicted correctly for 54.0% of coins in the bag


Here we are not doing much better than random guessing (or, you could say, coin tossing :D).

**Decreasing $\alpha$ makes us more hesitant to label a coin as biased:**

In [27]:
summarise_results(test_bag_of_coins(
    bag_of_coins,
    random_state=RandomState(17),
    alpha=0.001
))

actual,Biased,Fair
predicted,,
Biased,909,6
Fair,4091,4994


When we predict 'Fair', we are correct 55.0% of the time and incorrect 45.0% of the time
When we predict 'Biased', we are correct 99.3% of the time and incorrect 0.7% of the time
When the coin is 'Fair', we are correct 99.9% of the time and incorrect 0.1% of the time
When the coin is 'Biased', we are correct 18.2% of the time and incorrect 81.8% of the time
We predicted correctly for 59.0% of coins in the bag


Now we have high precision for identifying biased coins, but we pay for it in terms of low recall.

**Increasing $\alpha$ makes us more keen to label a coin as biased:**

In [28]:
summarise_results(test_bag_of_coins(
    bag_of_coins,
    random_state=RandomState(17),
    alpha=0.4
))

actual,Biased,Fair
predicted,,
Biased,4703,1894
Fair,297,3106


When we predict 'Fair', we are correct 91.3% of the time and incorrect 8.7% of the time
When we predict 'Biased', we are correct 71.3% of the time and incorrect 28.7% of the time
When the coin is 'Fair', we are correct 62.1% of the time and incorrect 37.9% of the time
When the coin is 'Biased', we are correct 94.1% of the time and incorrect 5.9% of the time
We predicted correctly for 78.1% of coins in the bag


Now we have high recall for identifying biased coins, but we pay for it in terms of lower precision.

**What happens if biased coins are very common in the bag?**

In [29]:
bag_of_coins_2 = (
      [0.5] * 1000   # 1000 fair coins
    + [0.6] * 4500   # 4500 coins biased towards heads
    + [0.4] * 4500   # 4500 coins biased towards tails
)

In [30]:
summarise_results(test_bag_of_coins(
    bag_of_coins_2,
    random_state=RandomState(17)
))

actual,Biased,Fair
predicted,,
Biased,5839,41
Fair,3161,959


When we predict 'Fair', we are correct 23.3% of the time and incorrect 76.7% of the time
When we predict 'Biased', we are correct 99.3% of the time and incorrect 0.7% of the time
When the coin is 'Fair', we are correct 95.9% of the time and incorrect 4.1% of the time
When the coin is 'Biased', we are correct 64.9% of the time and incorrect 35.1% of the time
We predicted correctly for 68.0% of coins in the bag


Here we seem to _over-predict_ the fair class; now **most of our predictions of fair are wrong**.

**What happens if biased coins are very rare in the bag?**

In [31]:
bag_of_coins_3 = (
      [0.5] * 9500  # 9500 fair coins
    + [0.6] * 250   # 250 coins biased towards heads
    + [0.4] * 250   # 250 coins biased towards tails
)

In [32]:
summarise_results(test_bag_of_coins(
    bag_of_coins_3,
    random_state=RandomState(17)
))

actual,Biased,Fair
predicted,,
Biased,323,399
Fair,177,9101


When we predict 'Fair', we are correct 98.1% of the time and incorrect 1.9% of the time
When we predict 'Biased', we are correct 44.7% of the time and incorrect 55.3% of the time
When the coin is 'Fair', we are correct 95.8% of the time and incorrect 4.2% of the time
When the coin is 'Biased', we are correct 64.6% of the time and incorrect 35.4% of the time
We predicted correctly for 94.2% of coins in the bag


What's really interesting here is that **most of the time when we predicted biased, we were wrong**.

**Setting $\alpha$ to a low value value like $0.05$ could not save us from this situation.**

Does that agree with your intuition? Or are you thinking something like

>"Hang on, setting $\alpha$ to $0.05$ means I should only be able to get it wrong one time in 20"?

**Food for thought:** This situation - with the thing we are looking for (biased coins) being rare - is **probably closer to what we have in science**. For example, for a given disease, the vast majority of chemical compounds are not an effective treatment.

[This fascinating paper](https://royalsocietypublishing.org/doi/10.1098/rsos.140216) argues that this is partly why "an alarming number of published results cannot be reproduced by other people", and does a series of simulations like I have done above.

## Homework question:

Of the various success and failure rates reported by `summarise_results`, which one(s) if any is/are fixed only by your setting of the threshold $\alpha$?

What do you think about that?

## Gratuitous quokka break:

![title](quokka_break_1.jpg)

## Hint: look at Bayes' rule:

If the above results are puzzling, one way to get intuition is to look at Bayes' rule:

$$P(\text{Fair} \;|\; \text{Observed}) = \frac{P(\text{Observed} \;|\; \text{Fair}) \cdot P(\text{Fair})}{P(\text{Observed})}$$

or equally

$$P(\text{Fair} \;|\; \text{Observed}) = \frac{P(\text{Observed} \;|\; \text{Fair}) \cdot P(\text{Fair})}
{(P(\text{Fair})\cdot P(\text{Observed} \;|\; \text{Fair}))
\;+\;
(P(\text{Biased})\cdot P(\text{Observed} \;|\; \text{Biased}))
}$$


The $p$-value reported by the significance test is only related to _one part_ of the RHS here, namely $P(\text{Observed} \;|\; \text{Fair})$; it doesn't reflect any of the other values that go into the RHS.

(Related, not equal, by the way, because the $p$-value is not the exact probability of the observed sequence of heads and tails, but instead the probability of any outcome equally or more extreme.)

## Concluding remark

In our work at Brandwatch, we won't be testing bags of coins, but we might be testing bags of:

- **keywords**: e.g. we want to know which keywords are or are not correlated with sentiment
- **times series**: e.g. we have the daily volume time series for a large number of queries, and we want to know which "really" show an increasing trend
- **pairs of survey variables**: e.g. we want to know which pairs of survey variables show a relationship that we can report as an "insight"

If we do mass significance testing of things like this, we will need to think carefully about what the results really tell us.

For example, simply stating "we're doing significance testing" doesn't free us from having to think about the two different kinds of errors we can make, how common these _actually_ are, and how they typically have different costs associated with them.


## Gratuitous quokka appendix:

![title](quokka_break_2.jpg)

## PS: $p$-values aren't measures of effect size!

The first coin here looks to have a much stronger bias, but the $p$-values come out bigger... (because the sample size is different)

In [33]:
num_heads = 60
num_tails = 30
print("Ratio of heads to tails: %.2f" % (num_heads/num_tails))
print("p-value: %.4f" % binom_test(x = [num_heads, num_tails]))

Ratio of heads to tails: 2.00
p-value: 0.0021


In [34]:
num_heads = 4000
num_tails = 3700
print("Ratio of heads to tails: %.2f" % (num_heads/num_tails))
print("p-value: %.4f" % binom_test(x = [num_heads, num_tails]))

Ratio of heads to tails: 1.08
p-value: 0.0007


**Message:** Don't try to compare effect sizes by comparing $p$-values when the sample sizes might be different.

**Example:** Suppose instead of coins we have keywords, and instead of heads and tails we have positive and negative mentions. Suppose we are trying to pick out which keyword has the most skewed sentiment. Sure, it might be a good idea to significance-test that there is a skew (not 50/50 positive and negative sentiment) so that we don't show the user keywords where any observed sentiment skew away from 50/50 is plausibly just due to chance.

But because different keywords can have radically different sample sizes in our data, it would be wrong to then go on to compare the $p$-values to see which keyword had the greater skew.